In [9]:
import pandas as pd

In [10]:
file_path = "/experiments/base_model/landmarks/embeddings.txt"

In [11]:
data = pd.read_csv(file_path, sep="\t") #sep=r"\,|\t", engine="python",header=0
#data.columns = ["a", "b", "c"]

In [12]:
data.columns

Index(['label', 'filename', 'embeddings'], dtype='object')

In [13]:
df = pd.DataFrame(data)

In [15]:
for i in range(0,65):
    print("'"+"E"+ str(i)+"'"+",",end="")

'E0','E1','E2','E3','E4','E5','E6','E7','E8','E9','E10','E11','E12','E13','E14','E15','E16','E17','E18','E19','E20','E21','E22','E23','E24','E25','E26','E27','E28','E29','E30','E31','E32','E33','E34','E35','E36','E37','E38','E39','E40','E41','E42','E43','E44','E45','E46','E47','E48','E49','E50','E51','E52','E53','E54','E55','E56','E57','E58','E59','E60','E61','E62','E63','E64',

In [16]:
df[['E0','E1','E2','E3','E4','E5','E6','E7','E8','E9','E10','E11','E12','E13','E14','E15','E16','E17','E18','E19','E20','E21','E22','E23','E24','E25','E26','E27','E28','E29','E30','E31','E32','E33','E34','E35','E36','E37','E38','E39','E40','E41','E42','E43','E44','E45','E46','E47','E48','E49','E50','E51','E52','E53','E54','E55','E56','E57','E58','E59','E60','E61','E62','E63']] = df['embeddings'].str.split(',',expand=True)

In [ ]:
Pose1_df, Pose2_df, Pose3_df, Pose4_df, Pose5_df, Pose6_df

Pose1, Pose2, Pose3, Pose4, Pose5, Pose6

In [25]:
Pose1_df = df[(df.label == 'Pose1')]
Pose1_df.to_pickle('dataframes/Pose1_df') 

In [26]:
Pose2_df = df[(df.label == 'Pose2')]
Pose2_df.to_pickle('dataframes/Pose2_df') 

In [27]:
Pose3_df = df[(df.label == 'Pose3')]
Pose3_df.to_pickle('dataframes/Pose3_df') 

In [28]:
Pose4_df = df[(df.label == 'Pose4')]
Pose4_df.to_pickle('dataframes/Pose4_df') 

In [29]:
Pose5_df = df[(df.label == 'Pose5')]
Pose5_df.to_pickle('dataframes/Pose5_df') 

In [30]:
Pose6_df = df[(df.label == 'Pose6')]
Pose6_df.to_pickle('dataframes/Pose6_df') 

In [31]:
Pose7_df = df[(df.label == 'Pose7')]
Pose7_df.to_pickle('dataframes/Pose7_df') 

# Check t-sne on Person 5

In [1]:
import argparse
import os
import json

import numpy as np
import tensorflow as tf

import model.create_dataset as create_dataset
from model.utils import Params
from model.model_fn import model_fn

In [22]:
model_dir = "experiments/base_model"
landmark_dir = "Images/images_old/person_05_jpg"

In [23]:
def _get_dataset(landmark_dir, params, class_dict_dir):

    dataset = create_dataset.dataset(landmark_dir, params, class_dict_dir)
    dataset = dataset.batch(params.batch_size)
    dataset = dataset.prefetch(1)

    return dataset


def _get_dataset_size(landmark_dir, image_type):

	size = 0
	for root, dirs, files in os.walk(landmark_dir):
	    files = [f for f in files if "."+image_type in f ]
	    size += len(files)

	tf.logging.info("Found {} {} landmarks in {}".format(size, image_type, landmark_dir))

	return size


def _get_embeddings(landmark_dir, estimator, params, class_dict_dir, landmark_size):
        
    # Compute embeddings
    tf.logging.info("Predicting on "+landmark_dir)    

    predictions = estimator.predict(lambda: _get_dataset(landmark_dir, params, class_dict_dir))

    embeddings = np.zeros((landmark_size, params.embedding_size))
    for i, p in enumerate(predictions):
        embeddings[i] = p['embeddings']

    tf.logging.info("Embeddings shape in "+os.path.basename(landmark_dir)+": {}".format(embeddings.shape))

    return embeddings

In [ ]:
if __name__ == '__main__':

    # Load the parameters from json file
    json_path = os.path.join(model_dir, 'params.json')
    assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
    params = Params(json_path)

    tf.reset_default_graph()
    tf.logging.set_verbosity(tf.logging.INFO)

    # Define the model
    tf.logging.info("Creating the model...")
    config = tf.estimator.RunConfig(tf_random_seed=230,
                                    model_dir=model_dir,
                                    save_summary_steps=params.save_summary_steps)
    estimator = tf.estimator.Estimator(model_fn, params=params, config=config)

    # Create a new folder to save embeddings
    embeddings_dir = os.path.join(model_dir, "landmarks")
    if not os.path.exists(embeddings_dir):
        os.makedirs(embeddings_dir)

    # Get the number of landmarks
    landmark_size = _get_dataset_size(os.path.normpath(landmark_dir), params.image_type)

    # Get embeddings and define tensorflow variables
    embeddings = _get_embeddings(landmark_dir, estimator, params, embeddings_dir, landmark_size)
    embeddings = np.round(embeddings, 6)                

In [25]:
embeddings

array([[ 1.8210e-03,  1.0500e-04, -2.2956e-02, ..., -4.2740e-02,
        -3.7427e-02, -3.2623e-02],
       [ 2.2690e-03,  6.2100e-04, -2.2293e-02, ..., -4.3060e-02,
        -3.7466e-02, -3.3096e-02],
       [ 1.6970e-03, -1.6500e-04, -2.3087e-02, ..., -4.2439e-02,
        -3.8011e-02, -3.2408e-02],
       ...,
       [ 1.6270e-03, -8.0000e-06, -2.3108e-02, ..., -4.2728e-02,
        -3.7968e-02, -3.2378e-02],
       [ 1.5980e-03,  2.3800e-04, -2.3256e-02, ..., -4.2621e-02,
        -3.8015e-02, -3.2385e-02],
       [ 1.3410e-03,  1.7100e-04, -2.3279e-02, ..., -4.2408e-02,
        -3.7709e-02, -3.2543e-02]])

In [26]:
import numpy as np
import os.path

class IdentityMetadata():
    def __init__(self, base, name, file):
        # dataset base directory
        self.base = base
        # identity name
        self.name = name
        # image file name
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
s='jpg'
a="jpeg"    
b="JPG" 
c="png" 
def load_metadata(path):
    metadata = []
    for i in os.listdir(path):
        #print(i)
        for f in os.listdir(os.path.join(path, i)):
             if f.endswith(s) or f.endswith(a)  or f.endswith(b) or f.endswith(c):
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

metadata = load_metadata(landmark_dir)
#print(metadata)
print(metadata.shape[0])

targets = np.array([m.name for m in metadata])

1162


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

%matplotlib inline
X_embedded = TSNE(n_components=2).fit_transform(embeddings)
#print(X_embedded)
fig=plt.figure(figsize=(15,8))
for i, t in enumerate(set(targets)):
    idx = targets == t
    plt.scatter(X_embedded[idx, 0], X_embedded[idx, 1], label=t)   

plt.legend(bbox_to_anchor=(1, 1));
fig.savefig("cluster_p5.jpg")

# Check t-sne on Person 1, 2, 3 and 5

In [28]:
model_dir = "experiments/base_model"
landmark_dir = "Images/p1_2_3_5"

In [ ]:
if __name__ == '__main__':

    # Load the parameters from json file
    json_path = os.path.join(model_dir, 'params.json')
    assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
    params = Params(json_path)

    tf.reset_default_graph()
    tf.logging.set_verbosity(tf.logging.INFO)

    # Define the model
    tf.logging.info("Creating the model...")
    config = tf.estimator.RunConfig(tf_random_seed=230,
                                    model_dir=model_dir,
                                    save_summary_steps=params.save_summary_steps)
    estimator = tf.estimator.Estimator(model_fn, params=params, config=config)

    # Create a new folder to save embeddings
    embeddings_dir = os.path.join(model_dir, "landmarks")
    if not os.path.exists(embeddings_dir):
        os.makedirs(embeddings_dir)

    # Get the number of landmarks
    landmark_size = _get_dataset_size(os.path.normpath(landmark_dir), params.image_type)

    # Get embeddings and define tensorflow variables
    embeddings = _get_embeddings(landmark_dir, estimator, params, embeddings_dir, landmark_size)
    embeddings = np.round(embeddings, 6)                

In [30]:
import numpy as np
import os.path

class IdentityMetadata():
    def __init__(self, base, name, file):
        # dataset base directory
        self.base = base
        # identity name
        self.name = name
        # image file name
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
s='jpg'
a="jpeg"    
b="JPG" 
c="png" 
def load_metadata(path):
    metadata = []
    for i in os.listdir(path):
        #print(i)
        for f in os.listdir(os.path.join(path, i)):
             if f.endswith(s) or f.endswith(a)  or f.endswith(b) or f.endswith(c):
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

metadata = load_metadata(landmark_dir)
#print(metadata)
print(metadata.shape[0])

targets = np.array([m.name for m in metadata])

2923


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

%matplotlib inline
X_embedded = TSNE(n_components=2).fit_transform(embeddings)
#print(X_embedded)
fig=plt.figure(figsize=(15,8))
for i, t in enumerate(set(targets)):
    idx = targets == t
    plt.scatter(X_embedded[idx, 0], X_embedded[idx, 1], label=t)   

plt.legend(bbox_to_anchor=(1, 1));
fig.savefig("cluster_p1_2_3_5.jpg")